In [1]:
# ==============================================================================
# Elastic Net – Thesis Tuning Pipeline
# ==============================================================================
# Spezifikation gemäß Masterarbeit:
# - Setup I (ifo only) vs. Setup II (ifo + Target Blocks) im Grid integriert.
# - FE: Lags 0-6 fix, SIS {300, 5k, 50k}, Redundanz {0.9, 1.0}, DR {None, PCA, PLS}.
# - Model: Mixing {0, 0.25, ..., 1}, Penalty {1e-4 ... 100}.
# ==============================================================================

import os, sys
from pathlib import Path
import numpy as np
import pandas as pd
from itertools import product

# --- 1) Pfad-Setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# --- 2) Imports ---
from src.config import (
    GlobalConfig,
    DEFAULT_CORR_SPEC,    # expanding
    EWMA_CORR_SPEC,       # ewma (lambda=0.98 in config default)
    outputs_for_model,
)
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import (
    load_target,
    load_ifo_features,
    load_full_lagged_features,
    load_rolling_importance,
)
from src.models.EN import ForecastModel

# --- 3) Konfiguration ---
USE_DYNAMIC_FI_PIPELINE = False  # False = Standard Setup (I & II)
SEED = 42

if USE_DYNAMIC_FI_PIPELINE:
    MODEL_NAME = "elastic_net_dynamic_fi"
else:
    MODEL_NAME = "elastic_net"

outputs_for_model(MODEL_NAME)
print(f"--- Starte Tuning für: {MODEL_NAME} ---")

# --- 4) Daten laden ---
y = load_target()             # Target: ΔIP
X_ifo = load_ifo_features()   # Features: ifo (2160 Spalten in Track 1)

# Align Indizes
idx_common = y.index.intersection(X_ifo.index)
y = y.loc[idx_common]
X_ifo = X_ifo.loc[idx_common]

# Platzhalter für Dynamic FI (werden nur geladen, wenn benötigt)
X_full_lagged = None
rolling_imp = None
y_fi = None

if USE_DYNAMIC_FI_PIPELINE:
    FI_BASE_DIR = PROJECT_ROOT / "outputs" / "feature_importance" / "outputs_no_missing"
    try:
        X_full_lagged = load_full_lagged_features(base_dir=FI_BASE_DIR)
        rolling_imp   = load_rolling_importance(base_dir=FI_BASE_DIR)

        # Intersection aller 3 Indizes
        idx_fi = y.index.intersection(X_full_lagged.index).intersection(rolling_imp.index)
        y_fi          = y.loc[idx_fi]
        X_full_lagged = X_full_lagged.loc[idx_fi]
        rolling_imp   = rolling_imp.loc[idx_fi]
        print(f"Dynamic FI Modus: {X_full_lagged.shape[1]} Features geladen.")
    except FileNotFoundError:
        print("FEHLER: Dynamic FI Artefakte nicht gefunden. Bitte feature_importance.ipynb ausführen.")
        sys.exit(1)
else:
    print(f"Full FE Modus (Setup I/II): {X_ifo.shape[1]} Basis-Features.")

# --- 5) Config Defaults (Thesis Policy) ---
def get_thesis_cfg() -> GlobalConfig:
    cfg = GlobalConfig(preset="thesis")
    # Explizite Policy-Parameter laut Thesis Setup
    cfg.policy_window = 24
    cfg.policy_decay = 0.97
    cfg.selection_mode = "decayed_best"
    return cfg

cfg_obj = get_thesis_cfg()

# --- 6) Grid Definition ---------------------------------------

def build_grid_full_fe():
    """
    Erstellt das Grid für Setup I (ifo) und Setup II (ifo + TargetBlocks).
    """
    # 1. Feature Engineering Optionen
    # Lags: Fix 0-6
    lag_candidates = [tuple(range(7))]

    # Korrelation
    corr_opts = [
        {"corr_spec": dict(DEFAULT_CORR_SPEC)},  # expanding
        {"corr_spec": dict(EWMA_CORR_SPEC)},     # ewma (lambda=0.98)
    ]

    # Screening (SIS) Top-K1
    k1_opts = [300, 5000, 50000] # 50k = "kein Screening"

    # Redundanzkontrolle (0.9 = stark, 1.0 = aus)
    red_opts = [0.9, 1.0]

    # Dimensionality Reduction (PCA/PLS mit 15/30)
    dr_opts = [
        {"dr_method": "none"},
        {"dr_method": "pca", "pca_kmax": 15, "pca_var_target": 0.99}, # var_target hoch setzen, damit kmax greift
        {"dr_method": "pca", "pca_kmax": 30, "pca_var_target": 0.99},
        {"dr_method": "pls", "pls_components": 15},
        {"dr_method": "pls", "pls_components": 30},
    ]

    # 2. Target Blocks (Setup Varianten)
    # None = Setup I, Listen = Setup II
    block_opts = [
        None,                                 # Setup I
        ["AR1", "Chronos"],                   # Setup II a
        ["AR1", "Chronos", "TSFresh"]         # Setup II b
    ]

    # 3. Sample Weighting
    weight_opts = [
        {"sample_weight_decay": None},
        {"sample_weight_decay": 0.99}
    ]

    # 4. Elastic Net Hyperparameter
    # alpha (Thesis) -> l1_ratio (sklearn)
    l1_ratios = [0.0, 0.25, 0.5, 0.75, 1.0]
    # lambda (Thesis) -> alpha (sklearn)
    alphas = [1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0]

    # --- Grid Assemblierung ---
    grid = []

    # Iteriere über FE-Kombinationen
    for lags, corr, k1, red, dr in product(lag_candidates, corr_opts, k1_opts, red_opts, dr_opts):

        # Plausibilitäts-Checks (Grid Thinning)
        # Wenn k1 klein ist (300), ist starke DR (PCA 30) evtl. redundant, aber wir lassen es laut Thesis drin.
        # Einzig logischer Cut: Wenn Redundanz=1.0 (aus) und k1=50000 (aus) und DR=None -> Modell explodiert.
        # ElasticNet kann High-Dim, aber es dauert. Wir lassen alles zu, das ist der Sinn der Thesis.

        base_fe = {
            "lag_candidates": lags,
            "k1_topk": k1,
            "redundancy_param": red,
            **dr,
            **corr
        }

        # Iteriere über Target Blocks & Gewichte
        for blocks, weights in product(block_opts, weight_opts):

            # Iteriere über Modell-HPs
            for l1, alpha in product(l1_ratios, alphas):
                hp = {
                    **base_fe,
                    "target_block_set": blocks,
                    **weights,
                    "alpha": l1,      # sklearn l1_ratio
                    "lambda": alpha,  # sklearn alpha
                    "seed": SEED
                }
                grid.append(hp)

    return grid

def build_grid_dynamic_fi():
    """
    Grid für Setup III (Dynamic Feature Importance).
    Hier kein FE-Tuning (passiert extern), nur Modell-HPs.
    """
    # Fix: Top Features gemäß Thesis (z.B. kumulativ 0.9 oder fix N)
    # Hier nehmen wir N=50 aus der config oder Thesis.
    # Thesis sagt: "Auswahl bis kumulierte Importance >= 0.9".
    # Tuning.py implementiert n_features_to_use (Top N).
    # Wir setzen dies hier auf einen vernünftigen Wert oder iterieren.

    n_features_list = [50] # Pragmatische Wahl für Grid

    weight_opts = [{"sample_weight_decay": None}, {"sample_weight_decay": 0.99}]
    l1_ratios = [0.0, 0.25, 0.5, 0.75, 1.0]
    alphas = [1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0]

    grid = []
    for n_feat, w, l1, alpha in product(n_features_list, weight_opts, l1_ratios, alphas):
        hp = {
            "n_features_to_use": n_feat,
            **w,
            "alpha": l1,
            "lambda": alpha,
            "seed": SEED
        }
        grid.append(hp)
    return grid

# --- 7) Ausführung --------------------------------------------

if USE_DYNAMIC_FI_PIPELINE:
    grid = build_grid_dynamic_fi()
    print(f"Dynamic FI Grid Größe: {len(grid)} Konfigurationen.")

    shortlist = run_stageA(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        model_grid=grid,
        X=X_ifo, # Dummy
        y=y_fi,
        cfg=cfg_obj,
        X_full_lagged=X_full_lagged,
        rolling_imp=rolling_imp
    )

    run_stageB(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        shortlist=shortlist,
        X=X_ifo, # Dummy
        y=y_fi,
        cfg=cfg_obj,
        X_full_lagged=X_full_lagged,
        rolling_imp=rolling_imp
    )

else:
    grid = build_grid_full_fe()
    print(f"Full FE Grid Größe (Setup I & II): {len(grid)} Konfigurationen.")

    # ACHTUNG: Das Grid kann sehr groß werden (~12k).
    # Für Stage A filtern wir aggressiv (Successive Halving in run_stageA).

    shortlist = run_stageA(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        model_grid=grid,
        X=X_ifo,
        y=y,
        cfg=cfg_obj,
        keep_top_k_final=10,
        min_survivors_per_block=0
    )

    run_stageB(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        shortlist=shortlist,
        X=X_ifo,
        y=y,
        cfg=cfg_obj
    )

print("\nTuning abgeschlossen.")



--- Starte Tuning für: elastic_net ---
INFO in load_ifo_features: Renaming columns to ensure validity.
Full FE Modus (Setup I/II): 13 Basis-Features.
Full FE Grid Größe (Setup I & II): 12600 Konfigurationen.
[Stage A] Using FULL FE (Gleis 1 & 2) pipeline.
[Stage A][Block 1] train_end=180, OOS=181-200 | configs=12600
  - Config 1/12600
    · Month 5/20 processed | running...RMSE=1.8220
    · Month 10/20 processed | running...RMSE=1.4479
    · Month 15/20 processed | running...RMSE=1.2859
    · Month 20/20 processed | running...RMSE=1.1402
  - Config 2/12600
    · Month 5/20 processed | running...RMSE=1.8113
    · Month 10/20 processed | running...RMSE=1.4374
    · Month 15/20 processed | running...RMSE=1.2803
    · Month 20/20 processed | running...RMSE=1.1348
  - Config 3/12600
    · Month 5/20 processed | running...RMSE=1.7661
    · Month 10/20 processed | running...RMSE=1.3938
    · Month 15/20 processed | running...RMSE=1.2572
    · Month 20/20 processed | running...RMSE=1.1137
  - 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.381e+02, tolerance: 3.195e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.385e+02, tolerance: 3.195e-02 Linear regression models with null weight for the l1 regularization term are mor

    · Month 5/20 processed | running...RMSE=1.5790
    · Month 10/20 processed | running...RMSE=1.2470
    · Month 15/20 processed | running...RMSE=1.1795
    · Month 20/20 processed | running...RMSE=1.0480
  - Config 7/12600
    · Month 5/20 processed | running...RMSE=1.6246
    · Month 10/20 processed | running...RMSE=1.2923
    · Month 15/20 processed | running...RMSE=1.2154
    · Month 20/20 processed | running...RMSE=1.0825
  - Config 8/12600
    · Month 5/20 processed | running...RMSE=1.8218
    · Month 10/20 processed | running...RMSE=1.4477
    · Month 15/20 processed | running...RMSE=1.2858
    · Month 20/20 processed | running...RMSE=1.1401
  - Config 9/12600
    · Month 5/20 processed | running...RMSE=1.8073
    · Month 10/20 processed | running...RMSE=1.4344
    · Month 15/20 processed | running...RMSE=1.2787
    · Month 20/20 processed | running...RMSE=1.1331
  - Config 10/12600
    · Month 5/20 processed | running...RMSE=1.7430
    · Month 10/20 processed | running...RMSE

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.569e+02, tolerance: 3.195e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.593e+02, tolerance: 3.195e-02 Linear regression models with null weight for the l1 regularization term are mor

    · Month 5/20 processed | running...RMSE=1.6196
    · Month 10/20 processed | running...RMSE=1.2555
    · Month 15/20 processed | running...RMSE=1.1712
    · Month 20/20 processed | running...RMSE=1.0421
  - Config 12/12600
    · Month 5/20 processed | running...RMSE=1.6100
    · Month 10/20 processed | running...RMSE=1.2728
    · Month 15/20 processed | running...RMSE=1.1988
    · Month 20/20 processed | running...RMSE=1.0677
  - Config 13/12600
    · Month 5/20 processed | running...RMSE=1.6377
    · Month 10/20 processed | running...RMSE=1.3059
    · Month 15/20 processed | running...RMSE=1.2268
    · Month 20/20 processed | running...RMSE=1.0938
  - Config 14/12600
    · Month 5/20 processed | running...RMSE=1.6377
    · Month 10/20 processed | running...RMSE=1.3059
    · Month 15/20 processed | running...RMSE=1.2268
    · Month 20/20 processed | running...RMSE=1.0938
  - Config 15/12600
    · Month 5/20 processed | running...RMSE=1.8215
    · Month 10/20 processed | running...R

...

    · Month 5/20 processed | running...RMSE=1.5368
    · Month 10/20 processed | running...RMSE=1.6183
    · Month 15/20 processed | running...RMSE=1.6180
    · Month 20/20 processed | running...RMSE=1.6322
  - Config 3296/12600


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.378e+01, tolerance: 3.195e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


    · Month 5/20 processed | running...RMSE=1.4736
    · Month 10/20 processed | running...RMSE=1.0786
    · Month 15/20 processed | running...RMSE=1.0527
    · Month 20/20 processed | running...RMSE=0.9406
  - Config 3297/12600


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.138e+02, tolerance: 3.195e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


KeyboardInterrupt: 